In [8]:
sides = [
    ({'spike_times': [20, 30, 40]},
    {'spike_times': [120, 130, 140]},
    {'spike_times': [220, 230, 240]}),
    ({'spike_times': [60, 70, 80]},
    {'spike_times': [160, 170, 180]},
    {'spike_times': [260, 270, 280]})

]
sides_sequence = [0, 1, 1, 0]

duration = 1000

#per ciascuna side
#per ciascun elemento nella lista spike_times
#mappa la funziona che prende i tempi di spike e li trasla di duration * i, mappata tante volte quante sono le side_sequence
#solo se la side_sequence è uguale all'indice della side
print(enumerate(sides))
new_spike_times = [
    [
        list(map(lambda x: x + duration * i, side['spike_times']))
        for i in range(len(sides_sequence))
        if sides_sequence[i] == j
    ]
    for j, side in enumerate(sides)
]

print(new_spike_times)


TypeError: tuple indices must be integers or slices, not str